In [27]:
import requests
from bs4 import BeautifulSoup
from io import StringIO
import re
import pandas as pd
import datetime
def get_scedule_for_n_weeks(week_in_fututre: int):
    """Finding the exam schema for n weeks in the future, with 0 being current.
    params: 
    week_in_fututre: int: How many weeks from now to get the exam scheme, wiht 0 being current
    -> pd.DataFrame with info"""
    
    # Setting the link for week 0
    week_0_response = requests.get('https://cloud.timeedit.net/itu/web/public/ri1Q56.html')
    soup = BeautifulSoup(week_0_response.content,"lxml")
    for i in range(week_in_fututre):
        # Finding the link for next week
        link_to_next_week = soup.find_all('a', {'aria-label' : "Go to next week"})[0]['href']
        # Get response for next week
        response = requests.get(link_to_next_week)
        # Make it to soup format
        soup = BeautifulSoup(response.content,"lxml")
        
    # When at the correct week
    schedule_link = soup.find('a', {'title' : 'CSV'}).get('href')
    response = requests.get(schedule_link)
    csv_thing = StringIO(response.text)
    return pd.read_csv(csv_thing,  skiprows = 3)
    

    
def parse_exams(exams):
    
    find_nr_physical_rooms = [len(re.findall("[0-9][A-X][0-9]{2}", room)) for room in exams.Room]
    any_physical_rooms = [bool(i) for i in find_nr_physical_rooms]
    nr_people = exams.apply(lambda x: len( x['Name,Person'].split(',')), axis = 1)
    # Find if there is a project code in the study activity
    exam['subject_code'] = [re.findall('[A-Z]{7}[0-9][A-Z]{2}|$', i)[0] for i in exams['Study Activity']]
    dont_recuire_subject_code = [False if i == 'Ordinary Exam' else True for i in exams['Exam']]
    
    have_required_subjectcode = [require or have for require, have in zip(dont_recuire_subject_code, subject_code)]
    
    
    ### TODO: Implement this if we want to exclude the duplicates
#     is_duplicate = exam_duplication(exams) 
    
    
    should_get_food = [code and room for code, room in zip(any_physical_rooms, have_required_subjectcode)]
    
    exams['people'] = nr_people
    exams['get_food'] = should_get_food
    exams
    exams.loc[len(exams.index)] = get_total(exams)
    return exams
    
def get_total(exams):
    

def find_orders(exams):
    food_order = exams.apply(find_one_order(), axis = 1)


    
def save_day(exams, comment, week, day):
    path = f'{folderpath}/{week}/day'
    print(comment, file=open(path + "-comment.txt", 'w'))
    exams.to_csv(path + "order.csv")
    
    
def create_comment(order):
    header = 'Order for exam: Bestilt af: SAP. '
    exam_comment = [Exam + order]
    
    
def create_orders(exams):
    create_folder("week")
    for day in exams['Begin date'].unique():
        exam_for_day = parse_exams(exams[exams['Begin date'] == day])
        exam_for_day = find_orders(exam_for_day)
        comment = create_comment(exam_for_day)
        save_day(exam_for_day, comment)
#     exam_code_if_needed = exams.apply(lambda x: 0 if x['Exam '] == 'Ordinary Exam' and '')

# current_week = datetime.datetime.now().isocalendar()[1] # Find current week

exams = get_scedule_for_n_weeks(2)  
exams

# per_day = parse_exams(exams.copy())
# per_day

,Begin date,Begin time,End date,End time,Exam,Study Activity,Room,"Name,Person",Programme,Comment
0,11.04.2023,15:00,11.04.2023,15:50,Thesis Exam,NaN,2A20,"Rasmus Reinhold Paulsen ,Stella Grasshof",SD - 2nd year,NaN
1,12.04.2023,09:00,12.04.2023,09:40,Project Exam,_NN,"2A18,Zoom","Olav W. Bertelsen,Pawel Rafal Grabarczyk",_NN,NaN
2,13.04.2023,10:10,13.04.2023,10:50,Project Exam,_NN,2F14,"Dan Witzner Hansen,Stella Grasshof",_NN,NaN
3,14.04.2023,11:00,14.04.2023,11:40,Project Exam,NaN,5A30,"Peter Nørgaard Petersen,Søren Skaarup",Master ILM,NaN
